This file presents several combinatorial reduction steps how a general target can be reduced to a manageable number of them.
Concretely, we have 1820 matrices, which satisfy the following conditions:
1. The elements of the matrix consists of 0 and 1
2. Each row is strictly larger than all rows below it, where the value of the row is evalued as a binary number

As there are 2^4 = 16 possible rows and each matrix is uniquely determined by choosing 4 out of these 16, there are exactly 16 choose 4 such matrices, which is exactly equal to 1820.

In this file , we aim to realize several actions on this set to reduce the sample size from 1820 to a more manageable 47.

In [1]:
# Required packages
import numpy as np

First we have some helper functions that can generate relevant lists of elements, but also gives us ways to compute their value (thinking of them as a binary expansion) and check whether they are ordered.

In [2]:
# Input: A natural number n  
# Output: A list with elements odered lists of length n with elements 0 and 1.
def ordered_list(n):
    result = []
    for index in range(1,n+1):
        result.append([1]*(index) + [0]*(n - index))
    return result
    

# Input: A natural number n  
# Output: A list with elements all lists of length n with elements 0 and 1.
def inductive_list(n):
    result = [[]]
    for index in range(n):
        result = [[0] + step for step in result] + [[1] + step for step in result]
    return result

# Input; A list with elements 0 and 1
# Output: A boolean determining whether it is ordered, meaning all 1s are in lower indeces
def is_ordered_list(unordered_list):
    if 0 in unordered_list and 1 in unordered_list:
        return max([index for index in range(len(unordered_list)) if unordered_list[index] == 1]) < min([index for index in range(len(unordered_list)) if unordered_list[index] == 0])
    return True

# Input: A list consisting of 0 and 1
# Output: The value of the list considered as a binary number
def list_value(input_list):
    input_list = list(reversed(input_list))
    return sum([input_list[index] * 2 ** index for index in range(len(input_list))])

There are two key reductions we can make here:
1. First of all we can assume that each column has at most two non-zero elements. If it is other-wise we can apply a NOT gate to achieve such a column.
2. We can assume that if a column has two ones, then the top is a one, again by applying a NOT gate if necessary.
3. We can assume that the columns are relatively ordered, by switching the position of the input wires.

We hence need helper functions that can check this condition.

In [3]:
# Input: A list of lists thought of as a matrix.
# Output: A natural number that is the maximum of the sum of the columns
def max_column_value(matrix_list):
    column_number = len(matrix_list[0])
    return max([sum([matrix_list[index_one][index_two] for index_one in range(len(matrix_list))]) for index_two in range(column_number)])


# Input: A list of lists thought of as a matrix.
# Output: A boolean determining whether the top of the column is 1 for every column which sums to 2.
def two_reflection_column(matrix_list):
    if max_column_value(matrix_list) < 2:
        return True 
    column_number = len(matrix_list[0])
    for index_two in range(column_number):
        if sum([matrix_list[index_one][index_two] for index_one in range(len(matrix_list))]) == 2:
            if matrix_list[0][index_two] == 0:
                return False
    return True
    
# Input: A matrix and a list of the same length
# Output: A boolean determining whether the lists satisfy the relative ordering condition.
def relative_ordering(matrix,list_two):
    change_index = [0]
    for list_one in matrix:
        for index in range(1,len(list_one)):
            if list_one[index] != list_one[index - 1] and index not in change_index:
                change_index.append(index)
    change_index.append(len(list_one))
    change_index.sort()
    for index in range(len(change_index)-1):
        if not is_ordered_list(list_two[change_index[index]:change_index[index+1]]):
                return False
    return True

Finally, we can generate all possible options.
1. Just generate any list of four different elements
2. Generate a list of four different elements, which satisfy the two reduction conditions mentioned above.

In [4]:
# Input: A natural number m
# Output: A list of all possible coordinates
def possible_coordinates_all(m):
    input_list = inductive_list(m)
    result_list = [[row] for row in input_list]
    for _ in range(m-1):
        result_list = [row_one + [row_two] for row_one in result_list for row_two in input_list if list_value(row_one[-1]) > list_value(row_two)]        
    return result_list, len(result_list)

# Input: A natural number m
# Output: A list of all classes of possible coordinates
def possible_coordinates_classes(m):
    input_list = inductive_list(m)
    result_list = [[row] for row in ordered_list(m)]
    for _ in range(m-1):
        result_list = [row_one + [row_two] for row_one in result_list for row_two in input_list if list_value(row_one[-1]) > list_value(row_two) and max_column_value(row_one + [row_two]) < 3 and two_reflection_column(row_one + [row_two]) and relative_ordering(row_one,row_two)]
    # for matrix in result_list:
    #     for check_index in range(1,len(matrix)):
    #         for index in range(len(check_index)):
    #             if not relative_ordering(row_one[index],row_two)
    # index in range(len(row_one)) relative_ordering(row_one[index],row_two)
    return result_list, len(result_list)

Let us do some examples and see how this manifests in practice.
We generate possible lists of four vectors with four coordinates.
We do both the case for all and the case of relevant reduction classes.

In [5]:
sample_count_all = possible_coordinates_all(4)[1]
sample_count_classes = possible_coordinates_classes(4)[1]
print("There are {} total cases, which reduce to {} cases.".format(sample_count_all, sample_count_classes))
#Technical comment: 1820 is precisely 16 choose 4.
print("Here are those {} cases:".format(sample_count_classes))
sample_list_classes = possible_coordinates_classes(4)[0]
for mat in sample_list_classes:
    print(np.array(mat))
    print('\n')


There are 1820 total cases, which reduce to 47 cases.
Here are those 47 cases:
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 0]]


[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 1]
 [0 0 0 0]]


[[1 0 0 0]
 [0 1 1 0]
 [0 0 0 1]
 [0 0 0 0]]


[[1 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]


[[1 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 0]]


[[1 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 1]]


[[1 1 0 0]
 [1 0 0 0]
 [0 0 1 1]
 [0 0 0 0]]


[[1 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 0 0]]


[[1 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]]


[[1 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 1]]


[[1 1 0 0]
 [1 0 0 0]
 [0 1 1 0]
 [0 0 0 0]]


[[1 1 0 0]
 [1 0 0 0]
 [0 1 1 0]
 [0 0 0 1]]


[[1 1 0 0]
 [1 0 0 0]
 [0 1 1 1]
 [0 0 0 0]]


[[1 1 0 0]
 [1 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]


[[1 1 0 0]
 [1 0 1 0]
 [0 1 0 0]
 [0 0 0 0]]


[[1 1 0 0]
 [1 0 1 0]
 [0 1 0 0]
 [0 0 0 1]]


[[1 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 0]]


[[1 1 0 0]
 [1 0 1 1]
 [0 1 0 0]
 [0 0 0 0]]


[[1 1 1 0]
 [1 0 0 0]
 [0 0 

The next step is to develop Quantum circuits for these 47 cases.